In [2]:
from astrodbkit import astrodb
from astropy.io import fits
from contextlib import redirect_stdout
import pandas as pd
import numpy as np
import re
import io
import os
import math
import csv
import pip

Dependencies = [
    'astrodbkit',
    'numpy',
    'pandas']


        ^+xw*"""^q_  0 p" F  F _F  p^^"___jM   j  F              F
          _,,__   q x" [  F J_ J  P  w""""_  _,"  9  _m^`"*____x"    _____
     v,,_w"   "M_ @ `, ",_!u_9__L F #  r^""^^"    f j"      "      _*"   "6_
         _,,__  B 9_ "v,_Zp*"""""^@u# P _m^"^u,a*"  j   ____       9       ""
       _F    `4 A_ "*-ap"            ^Lj" _smu,    _* ,-"   9_   _wf
     ^^"__,,_ jL  -- m<                5j! ____*-*^   &       """"     ___
       p"    9p`^u,av'     Columbia     `,*""""q_   _x" _aa,_        p^" ""u
     ,r  _____!L___,M   Astronomy Lab    Lsr--x_"^^`" qP     9      J      `M
       y^    "_    _J      Database      L_,,_ ?_    _#       ^v- -^"
      _F  __,_`^---"jr                  j___ ""y""^^""_,-r-u_
     r^ j!    ?s_, *"jp                g""""^q_b_    _F     "p      j^^""-w
        L  _,wma_  _x"jN__          __d"""^c  F  "-^""  _    "c____j'      L
       j" J"    """  _F 99Nu______g**L_""s  4 A,    _P"""^q_    ""         "-
     m^  j_  _-^""mw^" _' # 9"N""L ^, "s  b #   "--^"      0
          @ j"   _v-wa+" ," j  #  p  r j qF "q_   _*-wu_   9,     y^`"^w_
       _,!  0_  f   _m-**" _F _F  L _FjP ?,    "^""    "u    "---^      j
     """     # J   j"   p"""p-^ x^ p" d_   -q__a-mw_    j               `L
            V  `q  #   f   j   4   b   ^,   __      6_  ?,     _,__       "--
     *`^ww-"     F 9L_ b   1   4   `u_   "-^""*,    jh    ^-xm*"   9z
                )    0 `+a_ W__ 9,___"^^"+_     L   0                k
         _x-*v+"     9    0   "b    "_    "p   _0   `&    ___       d_


In [3]:
def install(package):
    pip.main(['install', package])

In [142]:
class append_database:
    """
    Columbia University Astronomy Lab database handler
    """
    def __init__(self):
        
        """This is the __init_ section of the class where initial constants, variables etc are declared and other
        initial namespace operations are performed"""
        
        
        #Schema of the Database
        
        self.database_schema = {
            
        #Keys are table names and values are column names within the tables  

            'SOURCES': ['id','ra','dec','designation','publication_shortname','names','components',
                        'companions','comments','version'],
            
            'CHANGELOG': ['id','date','user','machine_name','modified_tables','user_description','filename'],
            
            'IGNORE': ['id','id1','id2','tablename'],
            
            'MODES':['id','mode','publication_shortname'],
            
            'SYSTEMS': ['id','name','publication_shortname'],
            
            'PUBLICATIONS':['id','bibcode','shortname','DOI','description'],
            
            'INSTRUMENTS' : ['id', 'name', 'publication_shortname'],
            
            'TELESCOPES' : ['id','name','publication_shortname'],
            
            'DATA_REQUESTS' : ['id','source_id','source','data_request','purpose'],
            
            'VERSIONS' : ['id', 'version', 'data_published', 'publication_shortname', 'comment'],
            
            'PHOTOMETRY': ['id','source_id','band','magnitude','magnitude_unc','system_id','telescope_id',
                            'instrument_id', 'publication_shortname','epoch','comments','versions'],
            
            'SPECTRAL_TYPES' : ['id','source_id','spectral_type','spectral_type_unc','publication_shortname','adopted',
                                'comments','version','num_spt','num_spt_unc'],
            
            'PARALLAXES' : ['id','source_id','parallax','parallax_unc','publication_shortname','adopted','comments',
                            'version'],
            
            'PROPER_MOTIONS' : ['id', 'source_id', 'proper_motion_ra','proper_motion_ra_unc', 'proper_motion_dec',
                                'proper_motion_dec_unc','V_tan','V_tan_unc','publication_shortname',
                                'comments', 'versions'],
            
            'RADIAL_VELOCITIES' : ['id','source_id','radial_velocity','radial_velocity_unc','spectrum_id',
                                   'publication_shortname','comments','version'],
            
            'SPECTRA' : ['id', 'source_id', 'spectrum' ,'wavelength_units', 'flux_units' ,'wavelength_order',
                         'regime','publication_shortname','obs_date','instrument_id','telescope_id','mode_id',
                         'filename','comments','best','version','local_spectrum'],
            
            'COMPANIONS' : ['id','source_id','publication_shortname','type','n', 'status'],
            
            'ACTIVITY' : ['id','source_id','regime','line','wavelength_min','wavelength_max', 'frequency_min', 
                          'frequency_max','ew','ew_unc','l_lbol','l_lbol_unc','flux','flux_unc','publication_shortname',
                          'epoch','telescope_id','instrument_id','spectrum_id','comments'],
            
            'MEMBERSHIP' : ['id', 'source_id', 'publication_shortname', 'pmem', 'pmem_unc' ,'comments'],
            
            'MODELS' : ['id', 'model', 'publication_shortname'],
            
            'MASS' : ['id', 'source_id', 'mass', 'mass_unc', 'model_id', 'publication_shortname', 'comments'],
            
            'ROTATION' : ['id', 'source_id', 'publication_shortname', 'period', 'period_unc','telescope_id',
                          'instrument_id','comments','tau','tau_model_id','adopted']
            
        }
        
        
        #Dictonary where primary_ids are stored
        
        self.primary_id_dict = {
            
            'SOURCES': 0,
            'CHANGELOG': 0,
            'IGNORE' : 0,
            'MODES' : 0,
            'SYSTEMS' : 0,
            'PUBLICATIONS': 0,
            'INSTRUMENTS' : 0,
            'TELESCOPES': 0,
            'DATA_REQUESTS' : 0,
            'VERSIONS': 0,
            'PHOTOMETRY': 0,
            'SPECTRAL_TYPES' : 0,
            'PARALLAXES' : 0,
            'PROPER_MOTIONS' : 0,
            'RADIAL_VELOCITIES' : 0,
            'SPECTRA' : 0,
            'COMPANIONS': 0,
            'ACTIVITY' : 0,
            'MEMBERSHIP' : 0,
            'MODELS' : 0,
            'MASS' : 0,
            'ROTATION' : 0
            
        }
        
        
        for key in self.primary_id_dict:
            
            self.primary_id_dict[key] = self.get_index('tabledata/{}.sql'.format(key))
            
            
        self.source_id = 0
        
        #none value array            
        self.globalnulls = [-9999.0,-99.0,'no string','X','None','Nan','nan']
        
    # def remove none values using none value dict
    #def check mapping file is ok (check for ' )
    
    # add the latest row check for .sql creation
    
    # take backups
    
    #HELPER FUNCTIONS
    
    #creates an empty mapping file to be used when auto importing a fits file
    def create_mapping(self, filename = 'mapping.csv'):
        """Creates an unfilled csv file with column names of the database and empty rows that can be
        filled in order to map the .fits file into the csv. 
        """
        with open(filename, 'w') as csvfile:

            writer = csv.writer(csvfile)


            for key in database_schema:
                writer.writerow([key] + database_schema[key])


                for i in range(5):

                    writer.writerow('\n')
        
        print('Completed')
        
        
        
    #gets the highest index number from .sql data files
    def get_index(self,filename):
    
        try:
            for line in reversed(open(filename).readlines()):
                

                if '\n' == line:
                    continue
                    
                index = int(re.search('[0-9]+',line)[0])
                
                break
                
            return index

        except FileNotFoundError:
            return 0
        

    def write_row(self,tablename,valuesarray):
        
        """Writes a row of data values given in valuesarray into the given tablename
        
        Parameters
        ----------
        tablename: str
            name of the table in which the data will be inserted
                
        valuesarray: python array
            array of values to be inserted into the database
                
        Comments
        --------
        This function can be used to override the astrodbkit wrapper whenever it doesn't work.
        
        """
        self.primary_id_dict[tablename] += 1

        values = [str(i) for i in valuesarray]

        values[0] = str(self.primary_id_dict[tablename])
        
        
        if tablename != 'SOURCES':
            values[1] = str(self.source_id)

        if not os.path.exists('tabledata'):
            os.makedirs('tabledata')

            
        file = open('tabledata/{}.sql'.format(tablename.lower()),'a+')

        file.write("INSERT INTO '{}' VALUES\n".format(tablename))

        valuestowrite = '({});\n'.format(','.join(values))
        
        file.write(valuestowrite)

        file.close()

        
        

    #END HELPER FUNCTIONS
            
            
        """    def add_statics():
        """
        
        
    def add_statics(self):
        
        
        self.publications=[
                'dotter2008','delfosse2000','kraus2007','douglas2014','agueros2011','kovacs2014','kafka2006',
                'skrutskie2006','zacharias2012']
        
        self.telescopes = [ 
                'SDSS','2MASS','WISE','UKIRT','HST','Spitzer','IRTF','Keck I','Keck II','KPNO 4m',
                'KPNO 2.1m','KPNO Bok','MMT','CTIO 1.5m','CTIO 4m','Gemini North','Gemini South',
                'ARC 3.5m','Subaru','Magellan I Baade','Magellan II Clay','ESO 1m','CFHT','NTT',
                'Palomar 200-inch','Pan-STARRS 1','Palomar 60-inch','CTIO 0.9m','SOAR 4.1m','GTC',#30
                'ESO VLT U1','ESO VLT U2','ESO VLT U3','ESO VLT U4','MDM 2.4m','MDM 1.3m','WIYN 3.5m'
                'Mayall']
        
        self.instruments = [
                'R-C Spec','GMOS-N','GMOS-S','FORS','LRIS','SpeX','LDSS3','FOCAS','NIRSPEC','IRS',#10
                'FIRE','MagE','GoldCam','SINFONI','OSIRIS','Triplespec','X-Shooter','GNIRS','WIRCam','CorMASS',#20
                'ISAAC','IRAC','DIS','SuSI2','IRCS','NIRI','P1640','NIFS','STIS','CGS4'#30
                'NIRC','IRCAM','SPHERE','WFC3','SofI','WIRCam','NICI','PANIC','NEWFIRM','UIST',#40
                'MegaCam','UFTI','MIKE','GPI','NaCo','FourStar','Flamingos2','ModSpec','Hydra','SDSS-I/II',#50
                'APOGEE','BOSS','OSMOS']
        
        self.modes = [
                'Prism','SXD','LXD1.9','SL','LXD2.3', 'Echelle','LL','VPH Grism','Red Grism']
        
        self.systems = [
                'SDSS','2MASS','WISE','AB','ST','Vega','MKO','CIT','Johnson','Cousins',#10
                'Landolt','Stromgren','Bessel','GALEX','DENIS','APASS','Kepler','Gaia']
        
        self.models = [
                'dotter','dotter2008','delfosse2000','kraus2007']
        
        
        #CHECK FOR NEW MODEL NAMES?,
        
    def add_fits(self, fitsfile, csvmapfile):
        
        """ automatically populates the database from the fits file using the mapping.csv 
        file that links the two
        
        Parameters
        ----------
            fits_file: str 
                The name and path of the Flexible Image Transport System file
                
            cvsmapfile: str
                the name and path of the .csv file with a particular expected format that is used to map the 
                values in the fits file to database
        
        Constructs
        ----------
            .sql files that can be used by astrodbkit to add data into the database

        """
        
        
        # open the fits file and save the data segment into a variable
        fitsdata = fits.open(fitsfile)[1].data

        # open the .csv file for mapping and save into a variable
        mapping = pd.read_csv(csvmapfile,header=None,na_values='\n')
        
        #get the headers of the fits file
        fitscolumnnames = fitsdata.columns.names
        


        
        for datarow in fitsdata:
            
            
            #increment source_id which is unique for each .fits row
            self.source_id += 1
            
            
            for index,row in mapping.iterrows():
                
                
                #gate that switches to the current tablename based on the mapping.csv file
                if type(row[0]) != float:
                    
                    tablename = row[0]
                    
                    schema = self.database_schema[tablename.upper()]
                    
                    #continue to next column as the first column is just for table names
                    continue 
                
                    
                #The .csv file returns a float(nan) object as None so float means we are at a row with values.
                elif type(row[0]) == float:

                    values = ['NULL'] * len(schema)
                    servicearray = []

                    for i in range(len(row)):
                        
                        
                        if type(row[i]) == float:

                            if math.isnan(row[i]):
                                continue

                            else:
                                values[i-1] = str(row[i])

                        else:

                            split = row[i].split(',')

                            if row[i] in fitscolumnnames:
                                
                                servicearray.append(datarow[row[i]])
                                
                                try:
                                    float(str(datarow[row[i]]))
                                    values[i-1] = datarow[row[i]]

                                except ValueError:
                                    values[i-1] = "'{}'".format(datarow[row[i]])

                                    
                            elif len(split) > 1 and split[0] in fitscolumnnames :
                                
                                package = [str(datarow[header]) for header in split]

                                values[i-1] = "'{}'".format(','.join(package))

                            else:
                                values[i-1] =  "'{}'".format(row[i])

                                
                nullvalues = [i for i in servicearray if i in self.globalnulls]
                
                
                if values.count('NULL') < len(values) and len(nullvalues) != len(servicearray):
                    
                    
                    if tablename == 'SOURCES':
                                               
                        sourcenulls = [-9999.0,-99.0,'no string','X',0,-9999,'-9999','-9999.0','-99.0','0']
                        
                        valuestosplit = re.sub("'","",values[5])
                        sourcenamevalue = valuestosplit.split(',')

                        sourcenames = [ 'ASAS','','2MASS','SWASP','USNO','UCAC','GOLDMAN','EPIC']

                        zippedsourcenames = list(zip(sourcenames,sourcenamevalue))
                
                        values[5] = "'{}'".format(','.join(['{} {}'.format(i,j) for (i,j)\
                                                           in zippedsourcenames if j not in sourcenulls]))
                        
                        self.write_row(tablename,values) 
                        
                        
                    else:
                        
                        self.write_row(tablename,values) 

                else:
                    continue

            
            
            
        
        

In [143]:
row = 'ASAS_ID,HIP_ID,TWOMASSNAME,SWASP_ID,14,USNO_ID,UCAC_ID,GOLDMAN_SEQ,EPIC_ID'

In [147]:
systems=[
                'SDSS','2MASS','WISE','AB','ST','Vega','MKO','CIT','Johnson','Cousins',#10
                'Landolt','Stromgren','Bessel','GALEX','DENIS','APASS','Kepler','Gaia']

In [156]:
%timeit [(a+1,b) for a,b in list(enumerate(systems))]

100000 loops, best of 3: 4.04 µs per loop


In [158]:
%timeit enumerate(systems)

The slowest run took 6.71 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 276 ns per loop


In [145]:
type(re.match('[0-9]+',line)[0])

TypeError: 'NoneType' object is not subscriptable

In [151]:
pd.read_csv('mapping.csv')

,SOURCES,id,ra,dec,designation,publication_shortname,names,components,companions,comments,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,\n,NaN,RA,DEC,NaN,NaN,"ASAS_ID,HIP_ID,TWOMASSNAME,SWASP_ID,USNO_ID,UC...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CHANGELOG,id,date,user,machine_name,modified_tables,user_description,filename,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
dataclass = append_database()

dataclass.primary_id_dict

SOURCES
(786,82.7044069,1.9849469,NULL,NULL,'',NULL,NULL,NULL,NULL);

CHANGELOG
IGNORE
MODES
INSERT INTO "modes" VALUES(9,'Red Grism',NULL);

SYSTEMS
INSERT INTO "systems" VALUES(18,'Gaia',NULL);

PUBLICATIONS


INSERT INTO "publications" VALUES(19,NULL,'zacharias2012',NULL,NULL);

INSTRUMENTS
INSERT INTO "instruments" VALUES(51,'OSMOS',NULL);

TELESCOPES
INSERT INTO "telescopes" VALUES(37,'WIYN 3.5m',NULL);

DATA_REQUESTS
VERSIONS
PHOTOMETRY
(17475,786,NULL,8.163,0.256,NULL,NULL,NULL,'goldman2013',NULL,NULL,NULL);

SPECTRAL_TYPES
(3144,786,'no',NULL,NULL,NULL,'from the Hammer (interactive)',NULL,NULL,NULL);

PARALLAXES
(1541,786,15.2,1.5,'goldman2013',NULL,NULL,NULL);

PROPER_MOTIONS
(2855,786,11.49,3.9,NULL,NULL,NULL,NULL,'goldman2013',NULL,NULL);

RADIAL_VELOCITIES
SPECTRA
(139,719,'Nov_2012_specs/2012_11_13_finals_postcrash/06032727+240215.fits',NULL,NULL,NULL,NULL,NULL,NULL,'48','35',NULL,NULL,NULL,NULL,NULL,NULL);

COMPANIONS
(786,786,NULL,NULL,NULL,0.0);

ACTIVITY
(139,719,'opti

{'ACTIVITY': 139,
 'CHANGELOG': 0,
 'COMPANIONS': 786,
 'DATA_REQUESTS': 0,
 'IGNORE': 0,
 'INSTRUMENTS': 51,
 'MASS': 770,
 'MEMBERSHIP': 1568,
 'MODELS': 3,
 'MODES': 9,
 'PARALLAXES': 1541,
 'PHOTOMETRY': 17475,
 'PROPER_MOTIONS': 2855,
 'PUBLICATIONS': 19,
 'RADIAL_VELOCITIES': 0,
 'ROTATION': 893,
 'SOURCES': 786,
 'SPECTRA': 139,
 'SPECTRAL_TYPES': 3144,
 'SYSTEMS': 18,
 'TELESCOPES': 37,
 'VERSIONS': 0}

In [40]:


try:
    for line in reversed(open('tabledata/modes.sql').readlines()):
        index = int(re.sub('\(','',line.split(',')[0]))
        print(index)

except FileNotFoundError:
    print(0)

ValueError: invalid literal for int() with base 10: 'INSERT INTO "modes" VALUES9'

In [76]:
line = """INSERT INTO "modes" VALUES(19,'Red Grism',NULL);"""

In [46]:
re.search('INSERT',line)

<_sre.SRE_Match object; span=(0, 6), match='INSERT'>

In [90]:
re.search('[0-9]*',line)

<_sre.SRE_Match object; span=(0, 0), match=''>

In [33]:
dataclass.get_index('tabledata/ACTIVITY.sql')

139

In [17]:
dataclass.get_index('tabledata/activity.sql')

140

In [ ]:
split = row.split(',')

package = [str(datarow[header]) for header in split]

values[i-1] = "'{}'".format(','.join(package))

In [ ]:
if tablename == 'SOURCES':

    sourcenulls = [-9999.0,-99.0,'no string','X',0,-9999]

    valuestosplit = re.sub("'","",values[5])
    sourcenamevalue = valuestosplit.split(',')

    sourcenames = [ 'ASAS','','2MASS','SWASP','USNO','UCAC','GOLDMAN','EPIC']

    zippedsourcenames = list(zip(sourcenames,sourcenamevalue))

    values[5] = "'{}'".format(','.join(['{} {}'.format(i,j) for (i,j)\
                                       in zippedsourcenames if j not in sourcenulls]))

    self.write_row(tablename,values)
    
    

In [111]:
myclass = append_database()
myclass.add_fits('HyadesMembers.fits','mapping.csv')
hyadesdb = astrodb.Database('cualdatabase.sql', directory='tabledata')

In [179]:
def graph(data, colnames, titles=['X title','Y title'], width = 100, height = 500):

    zippedarray = []
    jsondataarray= []
    balloontextarray = []
 
    #generate JSON file from data which is a database query
    for line in data:
                
        line[0] = [i for i in line[0].split(',') if '2MASS' in i][0]
        
        zippedlist = list(zip(colnames,line))

        
        if -9999.0 in line or -00.0 in line or 0.0 in line or type(line[-1]) == type(None):
            continue
            
            
        for i,j in zippedlist:

            if type(j) == str or type(j) == np.str_:
                zippedarray.append('"{}":"{}"'.format(i,j))
            else:
                zippedarray.append('"{}":{}'.format(i,j))
            
            
        joinedline = ','.join(zippedarray)

        jsondataarray.append('{{{}}}'.format(joinedline))


    jsondata = '[{}]'.format(','.join(jsondataarray))
    #end of the json generation
    
    for colname in colnames:
        
        text = '{}:<b>[[{}]]</b>'.format(colname,colname)
        balloontextarray.append(text)
        
    balloontext = '<br>'.join(balloontextarray)
    
    
    
    graphmap = [('__DATA__', jsondata), ('__balloonText__',balloontext),('__xTitle__', titles[0]), 
                ('__yTitle__', titles[1]), ('__xName__', colnames[2]), ('__yName__', colnames[1]), 
                ('__width__', str(width)), ('__height__', str(height))]

    
    
    file = open('xygraphtemplate.html','r')
    filedata = file.read()
    
    
    for i,j in graphmap:
        
        filedata = re.sub(i,j,filedata)
        
        
        
    writefile = open('xygraph.html','w+')
    writefile.write(filedata)
    
    file.close()
    writefile.close()
    
    print('Completed')


In [177]:
data = hyadesdb.query("""SELECT 
                                 SOURCES.names,
                                 MASS.mass, 
                                 ROTATION.period,
                                 PHOTOMETRY.band, 
                                 PHOTOMETRY.magnitude,
                                 PHOTOMETRY.publication_shortname  FROM MASS
                                 
                               JOIN ROTATION  ON MASS.source_id = ROTATION.source_id 
                               JOIN SOURCES ON SOURCES.id = ROTATION.source_id
                               JOIN PHOTOMETRY ON PHOTOMETRY.source_id = ROTATION.source_id
                                                              
WHERE MASS.mass != -9999. AND 
ROTATION.period != -9999.0 AND 
ROTATION.period != -99.0 AND 
ROTATION.PERIOD != 0.0 AND
PHOTOMETRY.band = 'R' OR
PHOTOMETRY.band = 'K'""")

colnames = ['Obj ID','Period','Mass','Band','Magnitude','pub_short']
titles = ['Mass (Mo)','Period (p)']

array([ (' HIP20635,2MASS 04252215+2217381,UCAC 562-010189,GOLDMAN 309',  2.76086 ,  0., 'K',  4.077, None),
       (' HIP20635,2MASS 04252215+2217381,UCAC 562-010189,GOLDMAN 309',  2.76086 ,  0., 'K',  4.077, 'zacharias2012'),
       (' HIP21589,2MASS 04380945+1230388,UCAC 513-008381,GOLDMAN 473',  2.61056 ,  0., 'K',  4.105, None),
       ...,
       ('2MASS 06101775+2234199,UCAC 563-025173,GOLDMAN 723',  0.1977  ,  0., 'K',  9.044, 'zacharias2012'),
       ('2MASS 06131867+1122108,UCAC 507-021176,GOLDMAN 724',  0.581719,  0., 'K',  8.154, None),
       ('2MASS 06131867+1122108,UCAC 507-021176,GOLDMAN 724',  0.581719,  0., 'K',  8.154, 'zacharias2012')],
      dtype=[('\nsources.names', '<U144'), ('\nmass.mass', '<f8'), ('\nrotation.period', '<f8'), ('\nphotometry.band', '<U1'), ('\nphotometry.magnitude', '<f8'), ('\nphotometry.publication_shortname', 'O')])

In [174]:
[i for i in '2MASS 02145660+0347100,UCAC 469-002978,GOLDMAN 9'.split(',') if '2MASS' in i][0]

'2MASS 02145660+0347100'

In [169]:
hyadesdb.query('SELECT * FROM SOURCES WHERE names LIKE "%2 MASS%"')

In [180]:
# CODE TO GRAPH

graph(data,colnames,titles)

Completed


In [121]:
hyadesdb = astrodb.Database('cualdatabase.sql', directory='tabledata')

Populating database...
Loading changelog
Loading ignore
Loading modes
Loading systems
Loading publications
Loading instruments
Loading telescopes
Loading data_requests
Loading versions
Loading sources
Loading photometry
Loading spectral_types
Loading parallaxes
Loading proper_motions
Loading radial_velocities
Loading spectra
Loading companions
Loading activity
Loading membership
Loading models
Loading mass
Loading rotation
Database ready for use
